In [9]:
from PIL import Image
import nibabel as nb
import glob
import os
import numpy as np
from tqdm import tqdm

# Variables to change
file_names = glob.glob('/Volumes/atwai/Anderson_Scott/Example CFT Reconstructed ROI JP2 Stack/recon/ex_555nm_em_586nm/1500/*.tiff')
rgb_names = glob.glob('/Volumes/atwai/Anderson_Scott/Example CFT Reconstructed ROI JP2 Stack/recon/lowres/rgb/*.jpg')
ideal_dim = 512 #This is the number of pixels for the shorter edge

# Check if we are cropping the rgb image or not
rgb = (file_names[0] == rgb_names[0])

# Functions for cropping

def rolling_window(a, window):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)
           
def findFirst_numpy(a, b):
    temp = rolling_window(a, len(b))
    result = np.where(np.all(temp == b, axis=1))
    return result[0][0] if result else None

# First we load one sample image to create the desired dimensions

ref_image = Image.open(rgb_names[0])

# Find the boundaries to crop the image
# I have found 1.25e6 to be good delinator for the edge of the table

a = np.sum(np.sum(ref_image,2),0)>1.25e6
b = np.sum(np.sum(ref_image,2),1)>1.25e6

front = [0,0,0,1,1,1]
back = [1,1,1,0,0,0]

crop_dim0 = [findFirst_numpy(a, front), findFirst_numpy(a, back)]
crop_dim1 = [findFirst_numpy(b, front), findFirst_numpy(b, back)]

# Now we can resample the image to be smaller

# First we find the a rate to downsample the whole dataset by

width = crop_dim0[1] - crop_dim0[0]
length = crop_dim1[1] - crop_dim1[0]

downsample_rate = np.floor(min(width,length)/ideal_dim)

# Calculate the z downsample

z_included_slices = np.linspace(0, len(file_names)-1, num = int(len(file_names)/downsample_rate)).astype(int)

# Preallocate data

if rgb:
    new_img = np.zeros((int(length/downsample_rate), int(width/downsample_rate), 3, len(z_included_slices)))
else:
    new_img = np.zeros((int(length/downsample_rate), int(width/downsample_rate), len(z_included_slices)))

# Iterate through the slices

counter = 0
for zslice in tqdm(z_included_slices):
    
    #First we load the image
    image = Image.open(file_names[zslice])
    
    #Then we crop
    image = image.crop((crop_dim0[0],crop_dim1[0],crop_dim0[1], crop_dim1[1]))
    
    # Then we resize
    image = image.resize((int(width/downsample_rate), int(length/downsample_rate)))
    
    # Then we store it someplace
    if rgb:
        new_img[:,:,:,counter] = image
    else:
        new_img[:,:,counter] = image
        
    counter += 1
    
    # Delete the loaded image every time to preserve memory
    
    del image

# Determine the name for the new file
# This will be (name_of_the_folder)-stack-DownsampleRate-(downsamplerate).nii.gz
name = file_names[0].split('/')
prefix = name[-2]
name = name[0:-1]

# Calculate Downsample Rate for voxel volume
total_voxels = width*length*len(file_names) 
if rgb:
    new_voxels = new_img.shape[0]*new_img.shape[1]*new_img.shape[3]
else:
    new_voxels = new_img.shape[0]*new_img.shape[1]*new_img.shape[2]
    
downsample_volume_rate = np.round(total_voxels/new_voxels,2)

# Build the name
name[-1] = prefix+'-stack-DwnsmplVolRate-'+str(downsample_volume_rate)+'.nii.gz'
nifti_name = '/'+os.path.join(*name)
    
# Export to Nifti
nifti = nb.Nifti1Image(new_img,np.eye(4))
nb.save(nifti, nifti_name)

# Save a grayscale version as well
if rgb:
    name[-1] = 'grayscale-stack-DwnsmplVolRate-'+str(downsample_volume_rate)+'.nii.gz'
    nifti_name = '/'+os.path.join(*name)
    nifti = nb.Nifti1Image(np.mean(new_img,axis=2),np.eye(4))
    nb.save(nifti, nifti_name)
    
# Cleanup
del nifti, ref_image, rgb_names, file_names, new_img

print('Done')


100%|█████████████████████████████████████████| 347/347 [16:27<00:00,  2.85s/it]


Done


In [ ]:
rgb = nb.load('/Volumes/atwai/Anderson_Scott/Example CFT Reconstructed ROI JP2 Stack/recon/lowres/rgb-stack-DwnsmplVolRate-1009.nii.gz')
excitation = nb.load('/Volumes/atwai/Anderson_Scott/Example CFT Reconstructed ROI JP2 Stack/recon/ex_555nm_em_586nm/500-stack-DwnsmplVolRate-1009.41.nii.gz')

In [ ]:
rgb_img = rgb.get_fdata()
excitation_img = excitation.get_fdata()

In [ ]:
new_img = np.zeros((rgb_img.shape[0],rgb_img.shape[1],rgb_img.shape[3]))
#mask = float(excitation_img > np.max(np.ndarray.flatten(excitation_img))*0.8)
new_img = np.mean(rgb_img,axis=2)
    
nifti = nb.Nifti1Image(new_img,np.eye(4))
nb.save(nifti, '/Volumes/atwai/Anderson_Scott/grayscale.nii.gz')